In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os
import pandas as pd

In [ ]:
#Original
folder_list = sorted(os.walk("./result_12bit").__next__()[1])

lst_A = folder_list[1:5]
lst_B = folder_list[5:9]
lst_C = folder_list[9:13]

print(lst_A, lst_B, lst_C)

In [ ]:
#Window Setting Applied
path = './result_1/'
folder_list = sorted(os.walk(path).__next__()[1])

lst_A_2 = folder_list[0:4]
lst_B_2 = folder_list[4:8]
lst_C_2 = folder_list[8:12]

print(lst_A, lst_B, lst_C)

In [ ]:
def merge(lst, path):
    merged_gt = []
    merged_seg = []
    
    for folder in lst:
        gt_list = sorted(glob.glob(path + folder + '/GT*.jpg'))
        seg_list = sorted(glob.glob(path + folder + '/SEG*.jpg'))
        
        merged_gt += gt_list
        merged_seg += seg_list
    
    return merged_gt, merged_seg

In [ ]:
merged_A_gt, merged_A_seg = merge(lst_A, './result_12bit/')
merged_B_gt, merged_B_seg = merge(lst_B, './result_12bit/')
merged_C_gt, merged_C_seg = merge(lst_C, './result_12bit/')

#Change path
merged_A_gt_2, merged_A_seg_2 = merge(lst_A, path)
merged_B_gt_2, merged_B_seg_2 = merge(lst_B, path)
merged_C_gt_2, merged_C_seg_2 = merge(lst_C, path)


print(len(merged_C_gt), len(merged_C_gt_2))

In [ ]:
def DSC_list(gt_list, seg_list):
    
    DSC_list = []
    GT_sum = []
    
    for i in range(len(gt_list)):
        
        GT = cv2.imread(gt_list[i], 0) / 255

        GT[GT > 0.5] = 1
        GT[GT <= 0.5] = 0

        AUTO = cv2.imread(seg_list[i], 0) / 255

        AUTO[AUTO > 0.5] = 1
        AUTO[AUTO <= 0.5] = 0

        A_Intersect_G = np.multiply(AUTO, GT)
                
        DSC = (2 * np.sum(A_Intersect_G)) / (np.sum(AUTO) + np.sum(GT))

        DSC_list.append(DSC)
        GT_sum.append(np.sum(GT))
      
    return DSC_list


In [ ]:
DSC_list_A = DSC_list(merged_A_gt, merged_A_seg)
DSC_list_B = DSC_list(merged_B_gt, merged_B_seg)
DSC_list_C = DSC_list(merged_C_gt, merged_C_seg)

DSC_list_A_2 = DSC_list(merged_A_gt_2, merged_A_seg_2)
DSC_list_B_2 = DSC_list(merged_B_gt_2, merged_B_seg_2)
DSC_list_C_2 = DSC_list(merged_C_gt_2, merged_C_seg_2)


In [ ]:
print(len(DSC_list_A), len(DSC_list_A_2))

## Group Comparison

In [ ]:
window = '8-bit'

In [ ]:
# Original Image
gt_list = sorted(glob.glob('./result_12bit/512/GT*.jpg'))
seg_list = sorted(glob.glob('./result_12bit/512/SEG*.jpg'))
        
DSC_Original = DSC_list(gt_list, seg_list)


In [ ]:
gt_list_2 = sorted(glob.glob(path+'/512/GT*.jpg'))
seg_list_2 = sorted(glob.glob(path+'/512/SEG*.jpg'))
        
DSC_Original_2 = DSC_list(gt_list_2, seg_list_2)

In [ ]:
# Group A
bins = np.arange(0,1+0.01,0.01)

plt.hist(DSC_list_A, bins, label='Original Group A', histtype='step')
plt.hist(DSC_list_A_2, bins, label= window + ' Group A', histtype='step')

plt.xlabel('DSC Values')
plt.ylabel('Nodule Amount')
plt.title('Group A Original /'+ window + ' Comparison')
#plt.xlim(0.1,1)
plt.ylim(0,500)
plt.legend(loc='upper center')
plt.savefig('Group A Comparison', dpi=300)

In [ ]:
# Group B
bins = np.arange(0,1+0.01,0.01)

plt.hist(DSC_list_B, bins, label='Original Group B', histtype='step')
plt.hist(DSC_list_B_2, bins, label= window + ' Group B', histtype='step')

plt.xlabel('DSC Values')
plt.ylabel('Nodule Amount')
plt.title('Group B Original /'+ window + ' Comparison')
#plt.xlim(0.1,1)
plt.ylim(0,500)
plt.legend(loc='upper center')
plt.savefig('Group B Comparison', dpi=300)

In [ ]:
# Group C
bins = np.arange(0,1+0.01,0.01)

plt.hist(DSC_list_C, bins, label='Original Group C', histtype='step')
plt.hist(DSC_list_C_2, bins, label= window + ' Group C', histtype='step')

plt.xlabel('DSC Values')
plt.ylabel('Nodule Amount')
plt.title('Group C Original /'+ window + ' Comparison')
#plt.xlim(0.1,1)
plt.ylim(0,2000)
plt.legend(loc='upper center')
plt.savefig('Group C Comparison', dpi=300)

In [ ]:
# 512x512
bins = np.arange(0,1+0.01,0.01)

plt.hist(DSC_Original, bins, label='Original 512x512', histtype='step')
plt.hist(DSC_Original_2, bins, label= window +' 512x512', histtype='step')

plt.xlabel('DSC Values')
plt.ylabel('Nodule Amount')
plt.title('512x512 Original' + window + ' Comparison')
#plt.xlim(0.1,1)
plt.ylim(0,800)
plt.legend(loc='upper center')
plt.savefig('Original 512x512 Comparison', dpi=300)

## Comparison among Groups

In [ ]:
## Number of High, Low DSCs 
def DSC_count(DSC_list):
    high = 0
    low = 0
    for item in DSC_list:
        if item > 0.95:
            high += 1
        if item < 0.05:
            low +=1
    
    mean = np.mean(DSC_list)
            
    return high, low, mean

In [ ]:
def split(DSC_list, length):

    DSC_split = []
    j = 2013
    for i in range(length):
        
        DSC_split.append(DSC_list[j*i:j*(i+1)]) # Merge 된거 4개씩 나눠서 split에 넣음
    
    print(len(DSC_split))
    return DSC_split


In [ ]:
def Individual_Plot(DSC_list, DSC_list_2, sizes, group, length, type):
    
    DSC_split, DSC_split_2 = split(DSC_list, length), split(DSC_list_2, length)
    high_list, high_list_2 = [], []
    low_list, low_list_2 = [], []
    mean_list, mean_list_2 = [], []
    
    for i in range(len(DSC_split)):
        high, low, mean = DSC_count(DSC_split[i])
        high_list.append(high)
        low_list.append(low)
        mean_list.append(mean)
    
    for i in range(len(DSC_split_2)):
        high, low, mean = DSC_count(DSC_split_2[i])
        high_list_2.append(high)
        low_list_2.append(low)
        mean_list_2.append(mean)
    
    if type == 'low':
        df = pd.DataFrame({'Low:Original': low_list, 'Low:'+window: low_list_2}, index=sizes)
    else:
        df = pd.DataFrame({'High:Original': high_list, 'High:'+window: high_list_2}, index=sizes)
    
    ax = df.plot.bar(rot=0, width=0.5)
    
    plt.xlabel('Sizes')
    plt.ylabel('Nodule Amount')
    
    if type == 'low':
        plt.title('Number of Low DSCs Group '+group)
        plt.savefig('Bar_DSC_Low_'+group, dpi=300)
    else:
        plt.title('Number of High DSCs Group '+group)
        plt.savefig('Bar_DSC_High_'+group, dpi=300)
    
    

In [ ]:
DSC_merged = DSC_list_A + DSC_list_B + DSC_list_C + DSC_Original
DSC_merged_2 = DSC_list_A_2 + DSC_list_B_2 + DSC_list_C_2 + DSC_Original_2
lst_merged = lst_A + lst_B + lst_C + [512]
print(len(DSC_merged), lst_merged)

In [ ]:
Individual_Plot(DSC_merged, DSC_merged_2, lst_merged, 'All', 13, 'low')

In [ ]:
Individual_Plot(DSC_merged, DSC_merged_2, lst_merged, 'All', 13, 'high')

## Clearer Analysis 

In [ ]:
New_merged = DSC_list_A + DSC_list_B[:2013*2]
New_merged_2 = DSC_list_A_2 + DSC_list_B_2[:2013*2]
New_lst_merged = lst_A + lst_B[:2]
print(len(New_merged))

In [ ]:
Individual_Plot(New_merged, New_merged_2, New_lst_merged, '144~224', 6, 'low')

In [ ]:
Individual_Plot(New_merged, New_merged_2, New_lst_merged, '144~224', 6, 'high')